<a href="https://colab.research.google.com/github/TheRadDani/prefill-decoding-llms/blob/main/prefill_decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import torch
import torch.nn.functional as F

def generate(model, tokenizer, user_message="Tell me a joke.", max_tokens=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    chat = [{"role": "user", "content": user_message}]
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)

    past_key_values = None
    generated = []

    for _ in range(max_tokens):
        outputs = model(
            input_ids=input_ids,
            past_key_values=past_key_values,
            use_cache=True
        )

        logits = outputs.logits
        past_key_values = outputs.past_key_values

        next_token_logits = logits[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

        generated.append(next_token.item())
        input_ids = next_token  # Feed next token into the next decode step

        if next_token.item() == tokenizer.eos_token_id:
            break

    return tokenizer.decode(generated, skip_special_tokens=True)

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", trust_remote_code=True).to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [33]:
generate(model, tokenizer)

" It's about the number 8. I need to make sure it's appropriate for a kid. Also, I need to include a pun or double meaning in the joke. Maybe something with the number 8 and something else, like a cat or"